In [1]:
import subprocess, sys, os, glob

wkdir = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ'
os.chdir(wkdir)

### a method for run param formatted commands

In [2]:
def run_commds(command_with_param, successinfo, errorinfo):
    '''
    @param command_with_param: command with params to be run
    @param successinfo: info to be printed if command is successful
    @param errorinfo: info to be printed if command is unsuccessful
    '''

    print('run commands: {0}, on dir: {1}'.format(command_with_param, os.getcwd()))

    try:
        p = subprocess.Popen(
                            command_with_param,
                            shell=True,
                            cwd = os.getcwd(),
                            stdin = subprocess.PIPE, 
                            stdout = subprocess.PIPE, 
                            stderr = subprocess.PIPE) 

        output, error = p.communicate()

        output = output.decode("utf-8")

        if not output:
            output = successinfo

        if p.returncode == 0: 
            print('OUTPUT:\n {0}'.format(output)) 

            return True
        else: 
            print('ERROR:\n {0}'.format(error.decode("utf-8")))

            return False

    except Exception as e:
        print(e)
        print(errorinfo)

        return False


### bed files comes from many different results, their columns are different, they will be processed here
* remove GL KI and no chrxx started lines by grep -v -E 'GL|KI'
* cut -f1-6 to keep 6 columns
* then intersect or intersect -v after doing this.

In [ ]:
# need_to_cut_beds_path = '/Users/jplab/Desktop/snakepipes_Data/ATAC_seq_shFOXA1_snakepipes_results/MACS_allLen_noDUP_noChrM/naiveOverlap_q001/*.sorted.narrowPeak'
need_to_cut_beds_path = '/Users/jplab/Desktop/snakepipes_Data/ATAC_seq_shFOXA1_snakepipes_results/CSAW_MACS2/*_BDT/bed/*mostEnrich.sorted.bed'

need_to_cut_beds = glob.glob(need_to_cut_beds_path)

for f in need_to_cut_beds:
    ext_name = f.split('/')[-1].split('.')[-1]
    sample_name = f.split('/')[-1].split('.'+ext_name)[0]

    remove_GLKI_and_cut_commds = "cat {0} | grep -v -E 'GL|KI' | cut -f1-6 > {1}"
    remove_GLKI_and_cut_commds = remove_GLKI_and_cut_commds.format(f, sample_name+'.cut.temp.bed')
    
    if not run_commds(
        remove_GLKI_and_cut_commds, 
        'remove_GLKI_and_cut_commds successful', 
        'remove_GLKI_and_cut_commds unsuccessful'):
        print('fail on: {0}'.format(f))



### Bed files need to be sorted by position

In [3]:
# NC*.sorted.narrowPeak
# F_*.sorted.narrowPeak
navie_overlap_bed_file_path = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/F*_overlap_peaks.sorted.cut.temp.bed'
navie_overlap_bed_files = glob.glob(navie_overlap_bed_file_path)

most_enriched_bed_suffix = 'sh*mostEnrich.sorted.cut.temp.bed'
most_enriched_bed_path = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/' + most_enriched_bed_suffix
most_enriched_beds = glob.glob(most_enriched_bed_path)

n_bed = len(navie_overlap_bed_files)

if n_bed == 0:
    print('No bed files found,please provide correct bed files')
    sys.exit()

### get primary common overlap bed

In [4]:
query_bed = navie_overlap_bed_files[0]
other_bed = navie_overlap_bed_files[1:]

other_bed_str = ' '.join(other_bed)

# bed_intersect_command = 'bedtools intersect -wa -wb -a {0} -b {1} -filenames -sorted > common.bed'.format(query_bed, other_bed_str)
# just wa: query.bed, no wb other bed need to be reported, later will be easier to uniq lines.
# uniq -d: yes, just print duplicates lines(these lines are both overlap by A in B and C )
out_put_bed_name = 'primary_common.bed'
bed_intersect_command = 'bedtools intersect -wa -a {0} -b {1} -sorted | uniq -d  > '.format(query_bed, other_bed_str) + out_put_bed_name

if not run_commds(bed_intersect_command, 'bedtools intersect done', 'bedtools intersect failed'):
    print('fail on: {0}'.format(f))

run commands: bedtools intersect -wa -a /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/F_T_overlap_peaks.sorted.cut.temp.bed -b /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/F_D_overlap_peaks.sorted.cut.temp.bed /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/F_B_overlap_peaks.sorted.cut.temp.bed -sorted | uniq -d  > primary_common.bed, on dir: /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ
OUTPUT:
 bedtools intersect done


## process primary common overlap bed to final common overlap
* remove B14 most enrich but DMSO and TGF also open regions(both open: high low low), also TGF-beta

In [7]:
primary_common_overlap_bed_path = './' + out_put_bed_name

final_bed_name_temp = '{0}.bed'.format('final_common_overlap_temp')
final_bed_name = '{0}.bed'.format('final_common_overlap')
intersect_v_commands = 'bedtools intersect -a {0} -b {1} -v > {2}'

# change here
# commds = [intersect_v_commands.format(primary_common_overlap_bed_path, most_enriched_beds[0], final_bed_name_temp)]
commds = [intersect_v_commands.format(final_bed_name_temp, most_enriched_beds[1],final_bed_name)]

if not run_commds(commds, 'bedtools intersect done', 'bedtools intersect failed'):
    print('fail on: {0}'.format(f))

run commands: ['bedtools intersect -a final_common_overlap_temp.bed -b /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/shFOXA1_BDT_TGFmostEnrich.sorted.cut.temp.bed -v > final_common_overlap.bed'], on dir: /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ
OUTPUT:
 bedtools intersect done


## get uniq B14 or TGF-beta open region
* use B14 or TGF most enriched region to remove regions which open in common overlap region, to get uniq open region in B14 or TGF

In [10]:
final_common_overlap_bed_path = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/final_common_overlap.bed'

b14_most_enriched = '{0}.bed'.format('/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/shFOXA1_BDT_B14mostEnrich.sorted.cut.temp')
tgf_most_enriched = '{0}.bed'.format('/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/shFOXA1_BDT_TGFmostEnrich.sorted.cut.temp')
uniq_bed_name = 'shFOXA1_BDT_TGF'+'UniqOpen.bed'


intersect_v_commands = 'bedtools intersect -a {0} -b {1} -v > {2}'

# check args and check if matching are correct
commds = [intersect_v_commands.format(tgf_most_enriched, final_common_overlap_bed_path, uniq_bed_name)]

if not run_commds(commds, 'get uniq open region done', 'get uniq open region failed'):
    print('fail on: {0}'.format(f))

run commands: ['bedtools intersect -a /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/shFOXA1_BDT_TGFmostEnrich.sorted.cut.temp.bed -b /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/final_common_overlap.bed -v > shFOXA1_BDT_TGFUniqOpen.bed'], on dir: /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ
OUTPUT:
 get uniq open region done


### manually check

In [ ]:
# bedtools intersect \
# -a /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/ncFOXA1_BDT_B14mostEnrich.sorted.cut.temp.bed \
# -b /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/final_common_overlap.bed \
# -v \
# > ncFOXA1_BDT_TGFUniqOpen.bed

# bedtools intersect \
# -a /Users/jplab/Desktop/snakepipes_Data/ATAC_seq_shFOXA1_snakepipes_results/CSAW_MACS2/ncFOXA1_BDT/bed/ncFOXA1_BDT_B14mostEnrich.sorted.bed \
# -b /Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-20_ATAC_BDT_UNIQ/final_common_overlap.bed \
# -v \
# > ncFOXA1_BDT_B14UniqOpen.bed